In [1]:
from backend.vector_db.parent_embedd import create_parent_retriever

retriver = create_parent_retriever()

🔍 Creating ParentDocumentRetriever...


In [2]:
import os
import re
from typing import List

from dotenv import load_dotenv
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
child_collection_name = os.environ["CHILD_COLLECTION_NAME"]
def create_parent_retriever() -> ParentDocumentRetriever:
    """Creates parent retriver ParentDocumentRetriever"""

    client = QdrantClient()

    child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,
        chunk_overlap=60,
        length_function=len,
    )

    parent_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600 * 5,
        length_function=len,
        is_separator_regex=True,
        separators=["Art.[0-9]*"],
    )

    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=child_collection_name,
        embedding=NVIDIAEmbeddings(model=os.environ["EMBEDDER"]),
    )

    fs = LocalFileStore("./parent_docstore")
    docstore = create_kv_docstore(fs)

    print("🔍 Creating ParentDocumentRetriever...")
    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=docstore,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter
    )

    return retriever

In [3]:
retriever = create_parent_retriever()

🔍 Creating ParentDocumentRetriever...


In [6]:
from qdrant_client import QdrantClient, models
response, _ = retriever.vectorstore.client.scroll(collection_name="child_prawo_wodne",
    scroll_filter=models.Filter(
        should=[
            models.FieldCondition(
                key="metadata.article_number[]",
                match=models.MatchValue(value='1'),
            ),
        ],
    ),
    with_payload=True)

response

[Record(id='08b0ccda-eacc-4c38-bd7f-82ab01f0ebb5', payload={'page_content': '2011 r. – Prawo geologiczne i  górnicze (Dz.  U. z  2023 r. poz. 633) przepisów \nustawy nie stosuje się do: \n1) poszukiwania i rozpoznawania wód podziemnych; \n2) solanek, wód leczniczych oraz termalnych; \n3) wprowadzania do górotworu wód pochodzących z  odwodnienia zakładów \ngórniczych oraz wykorzystanych wód, o których mowa w pkt 2. \n                                                           \n5) dyrektywę 2006/118/WE Parlamentu Europejskiego i  Rady z  dnia 12  grudnia 2006  r. \nw sprawie ochrony wód podziemnych przed zanieczyszczeniem i pogorszeniem ich stanu', 'metadata': {'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-01-24T10:08:08+01:00', 'title': 'Akt prawny', 'author': 'Władysław Baksza', 'moddate': '2024-01-24T10:08:08+01:00', 'source': './data/prawod_wodne.pdf', 'total_pages': 444, 'page': 1, 'page_label': '2', 'article_number': ['1', '2', '3', '4

In [87]:
import json
def get_parent_elements(keys_list: List[str], fs: LocalFileStore) -> List[Document]:
    doc_list = []
    for key in keys_list:
        value = fs.mget([key])[0]
        decoded = value.decode("utf-8")
        data = json.loads(decoded)
        kwargs = data.get("kwargs")
        doc = Document(**kwargs)
        doc_list.append(doc)

    return doc_list

response, _ = retriever.vectorstore.client.scroll(
        collection_name=os.environ["CHILD_COLLECTION_NAME"],
        scroll_filter=models.Filter(
            should=[
                models.FieldCondition(
                    key="metadata.article_number[]",
                    match=models.MatchValue(value=str(194)),
                ),
            ],
        ),
        with_payload=True,
    )

parent_ids = []
for i in response:
	parent_element = (
		i.payload["metadata"]["doc_id"],
		i.payload["metadata"]["order"],
	)
	if parent_element not in parent_ids:
		parent_ids.append(parent_element)

parent_ids = [x[0] for x in sorted(parent_ids, key=lambda x: x[1], reverse=False)]

article_str = "\n\n".join(
	doc.page_content
	for doc in get_parent_elements(parent_ids, retriever.docstore.store)
)

In [90]:
print(article_str)

©Kancelaria Sejmu    s. 159/444 
 
2024-01-24 
 
4. Śródlądowe drogi wodne istotne dla zapewnienia zrównoważonego 
rozwoju systemu transportowego kraju są śródlądowymi drogami wodnymi 
o szczególnym znaczeniu transportowym. 
5. Śródlądowe drogi wodne o  szczególnym znaczeniu transportowym 
obejmują: 
1) śródlądowe drogi wodne istotne dla zapewnienia zrównoważonego rozwoju 
systemu transportowego kraju oraz grunty pod tymi drogami; 
2) nieruchomości gruntowe położone  w międzywalu śródlądowych dróg 
wodnych, o których mowa w pkt 1; 
3) nieruchomości, budynki i  budowle oraz urządzenia wodne, z  wyjątkiem 
wałów przeciwpowodziowych, funkcjonalnie i  bezpośrednio powiązane ze 
śródlądowymi drogami wodnymi, o  których mowa w  pkt 1, znajdujące się 
w obrębie ich działek ewidencyjnych. 
6. Śródlądowe drogi wodne o  szczególnym znaczeniu transportowym mogą 
podlegać budowie, przebudowie lub modernizacji. 
7. Rada Ministrów określi, w  drodze rozporządzenia, śródlądowe drogi 
wodne o szczegól

In [102]:
matches = re.finditer(r"Art.\s*[0-9]*", article_str)

start = -1
end = -1

for index, m in enumerate(matches):
    if start == -1 and m.group(0) == f'Art. {str(194)}':
        start = m.start()
    elif m.group(0) == f'Art. {str(195)}':
        end = m.start()
        break


In [103]:
for index, m in enumerate(matches):
    print(m)

<re.Match object; span=(11034, 11042), match='Art. 196'>


In [104]:
article_str[start:end]

'Art. 194. 1. Żeglugowe wykorzystanie śluz i  pochylni przez statki oraz \nwydatki ponoszone na śródlądowe drogi wodne i  ich infrastrukturę podlegają \newidencji. \n2. Wody Polskie prowadzą dla każdego roku kalendarzowego ewidencję: \n1) żeglugowego wykorzystania śluz i pochylni przez statki,\n\n©Kancelaria Sejmu    s. 160/444 \n \n2024-01-24 \n \n2) wydatków poniesionych na rozwój i utrzymanie śródlądowych dróg wodnych \ni ich infrastruktury \n– zgodnie z przepisami rozporządzenia Rady (EWG)  nr 1108/70 z dnia 4 czerwca \n1970 r. wprowadzającego system księgowy dla wydatków na infrastrukturę \nw transporcie kolejowym, drogowym i  w żegludze śródlądowej (Dz. Urz. WE L \n130 z 15.06.1970, str. 4, z późn. zm.7) – Dz. Urz. UE Polskie wydanie specjalne , \nrozdz. 7, t. 1, str. 42). \n3. Coroczne informacje o wydatkach poniesionych w roku poprzednim przez: \n1) zakłady lub jednostki organizacyjne, które z  własnych środków finansowych \nwykonują urządzenia wodne na śród lądowej drodze wodn

In [99]:
end

10892